In [1]:
import lib
import numpy as np
from scipy import special
import os

# Hartree Fock computation of the ground state of Quantum Dots

In this Jupyter Notebook we are going to study the groundstate of an N electron Quantum Dot. Our intention is to study how the chemical potential has jumps when shells are added (1s,2s2p). In both cases, the Self-Consistent Field method is used.

### Basis functions
First we write the basis we are going to use, i.e., the 3D eigenfunctions of the harmonic oscillator.

In [2]:
lz = 0.5
lx = 0.75

OMEGA_X = 1/lx**2
OMEGA_Z = 1/lz**2

def Hermite_pol(x, n):
    """
    Evaluates the Hermite polynomial of order n at x.

    Parameters
    ==========
    x : float or np.ndarray
        Values in which to evaluate the Hermite polynomial
    n : int
        Oder of the Hermite polynomial

    Returns
    =======
    float or np.ndarray
    """
    return special.eval_hermite(n, x)

def HO_wf(x, n, omega=OMEGA_X):
    """
    Evaluates the n^th eigenfunction of the Harmonic Oscillator at x.
    The units of x are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x : float or np.ndarray(N)
        Values in which to evaluate eigenfunction
    n : int
        Number of the eigenfunction
    omega : float
        Angular frequency of the Harmonic Oscillator

    Returns
    =======
    float or np.ndarray(N)
    """
    #return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) * np.exp(-omega*x**2 / 2) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))
    return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))

def HO_wf_3D(x, y, z, nx, ny, nz, omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z):
    """
    Evaluates the n^th eigenfunction of the (anisotropic) Harmonic Oscillator at x,y,z.
    The units of x,y,z are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x, y, z : float or np.ndarray(N)
        Position in which to evaluate the wave function
    nx, ny, nz : int
        Number of the eigenfunction for each cartesian coordinate
    omega_x, omega_y, omega_z : float
        Harmonic osciallation constant for each cartesian coordinate

    Returns
    =======
    float or np.ndarray(N)
    """
    return HO_wf(x, nx, omega=omega_x)*HO_wf(y, ny, omega=omega_y)*HO_wf(z, nz, omega=omega_z)

def index_to_q_numbers(k):
    """
    Returns the quantum numbers nx, ny, nz associated with the basis index k

    Parameters
    ----------
    k: int
        Index of the basis from 0 to 13

    Returns
    ----------
    nz ,ny, nz : int 
        Quantum numbers
    """

    q_numbers = np.array([(0,0,0),(0,0,1),(0,1,0),(1,0,0),(0,1,1),(1,0,1),(1,1,0),(1,1,1),(1,1,2),(1,2,1),(2,1,1),(1,2,2),(2,1,2),(2,2,1)])

    return q_numbers[k]

We now write the functions to calculate the integrands of the two-body integrals

In [3]:
def integrand_2(R, p, r, q, s):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    
    x1 = R[:,0]
    y1 = R[:,1]
    z1 = R[:,2]
    x2 = R[:,3]
    y2 = R[:,4]
    z2 = R[:,5]
    
    n_p = index_to_q_numbers(p-1)
    n_r = index_to_q_numbers(r-1)
    n_q = index_to_q_numbers(q-1)
    n_s = index_to_q_numbers(s-1)
    
    result = 1/r12
    
    result *= HO_wf_3D(x1, y1, z1, n_p[0], n_p[1], n_p[2])
    result *= HO_wf_3D(x2, y2, z2, n_r[0], n_r[1], n_r[2])
    result *= HO_wf_3D(x1, y1, z1, n_q[0], n_q[1], n_q[2])
    result *= HO_wf_3D(x2, y2, z2, n_s[0], n_s[1], n_s[2])
  
    
    return result

def norm_product(p, r, q, s):
    return (np.pi/OMEGA_X)**2 * np.pi/OMEGA_Z

def two_body_integrand(p, r, q, s):
    f = lambda R: integrand_2(R, p, r, q, s)*norm_product(p, r, q, s)
    
    return f

### One- and two- body integral computations
We first initialize some necessary parameters

In [23]:
N_electrons = 2
N_basis = 8
N_points = 100000
integrals_file = "integrals_QD.npy"

normalized_wf = True

max_iter_SCF = 500
eps_SCF = 0.2
Delta_SCF = 0

The exact one-body integral function is

In [5]:
def analytical_1(p,q):
    n_p = index_to_q_numbers(p-1)
    I = (p==q)*(OMEGA_X*(n_p[0] + n_p[1] + 1) + OMEGA_Z*(n_p[2] + 0.5))
    
    return I

The covariance function for the exponents of the gaussians is

In [6]:
def f_cov(p, r, q, s):
    
    cov = 0.5*np.diag([1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z, 1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z])
    
    return cov

We then calculate and store the one- and two-body integrals using lib.py

In [7]:
# One- and Two-body integrals
MC_args = {"f_cov":f_cov, "f_integrand":two_body_integrand, "N_points":N_points}
integrals = lib.integral_master(N_basis)
integrals.calculate(integrals_file, analytical_1, MC_args = MC_args)

Integral file already exists. Not computing the integrals. 


We calculate the overlap matrix S

In [8]:
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(ALPHA[p]+ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
# print('S',S)

Finally, we run the Self-Consistent Field for Hartree Fock

In [24]:
lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)

E = 6.0444444 | N(SCF) = 1
E = 8.1772596 | N(SCF) = 2
E = 8.6505117 | N(SCF) = 3
E = 9.1748453 | N(SCF) = 4
E = 8.5075556 | N(SCF) = 5
E = 8.6050411 | N(SCF) = 6
E = 8.6568397 | N(SCF) = 7
E = 8.6313023 | N(SCF) = 8
E = 8.6211703 | N(SCF) = 9
E = 8.6361078 | N(SCF) = 10
E = 8.6250187 | N(SCF) = 11
E = 8.6328183 | N(SCF) = 12
E = 8.6248225 | N(SCF) = 13
E = 8.6322140 | N(SCF) = 14
E = 8.6245076 | N(SCF) = 15
E = 8.6312917 | N(SCF) = 16
E = 8.6243619 | N(SCF) = 17
E = 8.6305003 | N(SCF) = 18
E = 8.6242043 | N(SCF) = 19
E = 8.6298308 | N(SCF) = 20
E = 8.6240720 | N(SCF) = 21
E = 8.6292713 | N(SCF) = 22
E = 8.6239612 | N(SCF) = 23
E = 8.6288144 | N(SCF) = 24
E = 8.6238704 | N(SCF) = 25
E = 8.6284462 | N(SCF) = 26
E = 8.6237977 | N(SCF) = 27
E = 8.6281525 | N(SCF) = 28
E = 8.6237407 | N(SCF) = 29
E = 8.6279197 | N(SCF) = 30
E = 8.6236970 | N(SCF) = 31
E = 8.6277360 | N(SCF) = 32
E = 8.6236641 | N(SCF) = 33
E = 8.6275913 | N(SCF) = 34
E = 8.6236400 | N(SCF) = 35
E = 8.6274773 | N(SCF) = 36
E

E = 8.6236208 | N(SCF) = 305
E = 8.6270183 | N(SCF) = 306
E = 8.6236208 | N(SCF) = 307
E = 8.6270183 | N(SCF) = 308
E = 8.6236208 | N(SCF) = 309
E = 8.6270183 | N(SCF) = 310
E = 8.6236208 | N(SCF) = 311
E = 8.6270183 | N(SCF) = 312
E = 8.6236208 | N(SCF) = 313
E = 8.6270183 | N(SCF) = 314
E = 8.6236208 | N(SCF) = 315
E = 8.6270183 | N(SCF) = 316
E = 8.6236208 | N(SCF) = 317
E = 8.6270183 | N(SCF) = 318
E = 8.6236208 | N(SCF) = 319
E = 8.6270183 | N(SCF) = 320
E = 8.6236208 | N(SCF) = 321
E = 8.6270183 | N(SCF) = 322
E = 8.6236208 | N(SCF) = 323
E = 8.6270183 | N(SCF) = 324
E = 8.6236208 | N(SCF) = 325
E = 8.6270183 | N(SCF) = 326
E = 8.6236208 | N(SCF) = 327
E = 8.6270183 | N(SCF) = 328
E = 8.6236208 | N(SCF) = 329
E = 8.6270183 | N(SCF) = 330
E = 8.6236208 | N(SCF) = 331
E = 8.6270183 | N(SCF) = 332
E = 8.6236208 | N(SCF) = 333
E = 8.6270183 | N(SCF) = 334
E = 8.6236208 | N(SCF) = 335
E = 8.6270183 | N(SCF) = 336
E = 8.6236208 | N(SCF) = 337
E = 8.6270183 | N(SCF) = 338
E = 8.6236208 

E = 8.6270183 | N(SCF) = 138
E = 8.6236208 | N(SCF) = 139
E = 8.6270183 | N(SCF) = 140
E = 8.6236208 | N(SCF) = 141
E = 8.6270183 | N(SCF) = 142
E = 8.6236208 | N(SCF) = 143
E = 8.6270183 | N(SCF) = 144
E = 8.6236208 | N(SCF) = 145
E = 8.6270183 | N(SCF) = 146
E = 8.6236208 | N(SCF) = 147
E = 8.6270183 | N(SCF) = 148
E = 8.6236208 | N(SCF) = 149
E = 8.6270183 | N(SCF) = 150
E = 8.6236208 | N(SCF) = 151
E = 8.6270183 | N(SCF) = 152
E = 8.6236208 | N(SCF) = 153
E = 8.6270183 | N(SCF) = 154
E = 8.6236208 | N(SCF) = 155
E = 8.6270183 | N(SCF) = 156
E = 8.6236208 | N(SCF) = 157
E = 8.6270183 | N(SCF) = 158
E = 8.6236208 | N(SCF) = 159
E = 8.6270183 | N(SCF) = 160
E = 8.6236208 | N(SCF) = 161
E = 8.6270183 | N(SCF) = 162
E = 8.6236208 | N(SCF) = 163
E = 8.6270183 | N(SCF) = 164
E = 8.6236208 | N(SCF) = 165
E = 8.6270183 | N(SCF) = 166
E = 8.6236208 | N(SCF) = 167
E = 8.6270183 | N(SCF) = 168
E = 8.6236208 | N(SCF) = 169
E = 8.6270183 | N(SCF) = 170
E = 8.6236208 | N(SCF) = 171
E = 8.6270183 

E = 8.6236208 | N(SCF) = 449
E = 8.6270183 | N(SCF) = 450
E = 8.6236208 | N(SCF) = 451
E = 8.6270183 | N(SCF) = 452
E = 8.6236208 | N(SCF) = 453
E = 8.6270183 | N(SCF) = 454
E = 8.6236208 | N(SCF) = 455
E = 8.6270183 | N(SCF) = 456
E = 8.6236208 | N(SCF) = 457
E = 8.6270183 | N(SCF) = 458
E = 8.6236208 | N(SCF) = 459
E = 8.6270183 | N(SCF) = 460
E = 8.6236208 | N(SCF) = 461
E = 8.6270183 | N(SCF) = 462
E = 8.6236208 | N(SCF) = 463
E = 8.6270183 | N(SCF) = 464
E = 8.6236208 | N(SCF) = 465
E = 8.6270183 | N(SCF) = 466
E = 8.6236208 | N(SCF) = 467
E = 8.6270183 | N(SCF) = 468
E = 8.6236208 | N(SCF) = 469
E = 8.6270183 | N(SCF) = 470
E = 8.6236208 | N(SCF) = 471
E = 8.6270183 | N(SCF) = 472
E = 8.6236208 | N(SCF) = 473
E = 8.6270183 | N(SCF) = 474
E = 8.6236208 | N(SCF) = 475
E = 8.6270183 | N(SCF) = 476
E = 8.6236208 | N(SCF) = 477
E = 8.6270183 | N(SCF) = 478
E = 8.6236208 | N(SCF) = 479
E = 8.6270183 | N(SCF) = 480
E = 8.6236208 | N(SCF) = 481
E = 8.6270183 | N(SCF) = 482
E = 8.6236208 

E = 8.6270183 | N(SCF) = 270
E = 8.6236208 | N(SCF) = 271
E = 8.6270183 | N(SCF) = 272
E = 8.6236208 | N(SCF) = 273
E = 8.6270183 | N(SCF) = 274
E = 8.6236208 | N(SCF) = 275
E = 8.6270183 | N(SCF) = 276
E = 8.6236208 | N(SCF) = 277
E = 8.6270183 | N(SCF) = 278
E = 8.6236208 | N(SCF) = 279
E = 8.6270183 | N(SCF) = 280
E = 8.6236208 | N(SCF) = 281
E = 8.6270183 | N(SCF) = 282
E = 8.6236208 | N(SCF) = 283
E = 8.6270183 | N(SCF) = 284
E = 8.6236208 | N(SCF) = 285
E = 8.6270183 | N(SCF) = 286
E = 8.6236208 | N(SCF) = 287
E = 8.6270183 | N(SCF) = 288
E = 8.6236208 | N(SCF) = 289
E = 8.6270183 | N(SCF) = 290
E = 8.6236208 | N(SCF) = 291
E = 8.6270183 | N(SCF) = 292
E = 8.6236208 | N(SCF) = 293
E = 8.6270183 | N(SCF) = 294
E = 8.6236208 | N(SCF) = 295
E = 8.6270183 | N(SCF) = 296
E = 8.6236208 | N(SCF) = 297
E = 8.6270183 | N(SCF) = 298
E = 8.6236208 | N(SCF) = 299
E = 8.6270183 | N(SCF) = 300
E = 8.6236208 | N(SCF) = 301
E = 8.6270183 | N(SCF) = 302
E = 8.6236208 | N(SCF) = 303
E = 8.6270183 

E = 8.6236208 | N(SCF) = 91
E = 8.6270183 | N(SCF) = 92
E = 8.6236208 | N(SCF) = 93
E = 8.6270183 | N(SCF) = 94
E = 8.6236208 | N(SCF) = 95
E = 8.6270183 | N(SCF) = 96
E = 8.6236208 | N(SCF) = 97
E = 8.6270183 | N(SCF) = 98
E = 8.6236208 | N(SCF) = 99
E = 8.6270183 | N(SCF) = 100
E = 8.6236208 | N(SCF) = 101
E = 8.6270183 | N(SCF) = 102
E = 8.6236208 | N(SCF) = 103
E = 8.6270183 | N(SCF) = 104
E = 8.6236208 | N(SCF) = 105
E = 8.6270183 | N(SCF) = 106
E = 8.6236208 | N(SCF) = 107
E = 8.6270183 | N(SCF) = 108
E = 8.6236208 | N(SCF) = 109
E = 8.6270183 | N(SCF) = 110
E = 8.6236208 | N(SCF) = 111
E = 8.6270183 | N(SCF) = 112
E = 8.6236208 | N(SCF) = 113
E = 8.6270183 | N(SCF) = 114
E = 8.6236208 | N(SCF) = 115
E = 8.6270183 | N(SCF) = 116
E = 8.6236208 | N(SCF) = 117
E = 8.6270183 | N(SCF) = 118
E = 8.6236208 | N(SCF) = 119
E = 8.6270183 | N(SCF) = 120
E = 8.6236208 | N(SCF) = 121
E = 8.6270183 | N(SCF) = 122
E = 8.6236208 | N(SCF) = 123
E = 8.6270183 | N(SCF) = 124
E = 8.6236208 | N(SCF) 

KeyboardInterrupt: 

In [14]:

# Este bloque habrá que acabar quitándolo
cov = f_cov(1,1,2,2)
integrand = two_body_integrand(1,1,2,2)

N_points = 100000
dim = cov.shape[0]
for i in range(10):
    steps = np.random.multivariate_normal(np.zeros(dim), cov, N_points) 

    I_avg= np.average(integrand(steps))
    #print(norm_product(1,1,1,1))

    print(I_avg)


0.16247090551927373
0.16133363777993243
0.1545630192515409
0.16071213547189636
0.15171139753118018
0.15790618497393233
0.15932009569130196
0.15616132551176265
0.16023981081168343
0.16036522631838035
